# Index Replication

We are given $N$ stocks and an index.

Strategy: At rebalancing time $t$, we use return information over the period from $\left[t-obs, t-1\right]$ to compute portfolio allocations $w_t$. We hold this portfolio over the period $\left[t,t+hold\right]$.

Let $r_s^p = \sum_jw_t^jr_{s}^j, s = t+1, t+2, \ldots, t+hold$ be portfolio returns and $r_s^I$ be the corresponding index returns. We explore several "closeness" measures:

1. mean squared error (MSE)
$$\frac{1}{hold}\sum_s \vert r_s^p - r_s^I \vert^2$$

2. maximum squared error (MXSE)
$$\max_s \vert r_s^p - r_s^I \vert^2$$

3. end of period wealth squared error (EOPSE)
$$\left(\prod_s(1+r_{s}^p) - \prod_s(1+r_{s}^I)\right)^2$$

4. maximum absolute deviation (MAD) 
$$\max_s \vert r_s^p - r_s^I \vert$$

In [70]:
using Flux, CSV, DataFrames, LinearAlgebra, Random, BSON, Statistics
using TensorBoardLogger, Logging, ProgressMeter

## Data

In [4]:
df = CSV.read("hsi_rets.csv",DataFrame)
dropmissing!(df)
tid = df.Date
idx = df[:,2] |> Vector{Float32}
stx = df[:,3:end] |> Matrix{Float32} |> transpose
size(idx), size(stx)

((3227,), (53, 3227))

In [26]:
N, T = size(stx)
obs, hold = 30, 30
tcost = 0.005

0.005

### Features

Features are $N \times obs$ matrices. We treat them as single channel images, so we reshape them to $N \times obs \times 1 \times B$, where B is the batch size.

In [6]:
rng = obs+1:hold:T-hold
ntrain = 70

70

In [7]:
xs = [stx[:,t-obs:t+hold] for t ∈ rng]
ys = [idx[t+1:t+hold] for t ∈ rng];

Shuffle and split into train and test sets.

In [8]:
ids = collect(1:length(xs))
shuffle!(ids)
xtrain, xtest = xs[ids[1:ntrain]], xs[ids[ntrain+1:end]]
ytrain, ytest = ys[ids[1:ntrain]], ys[ids[ntrain+1:end]];

## Loss Functions

In [9]:
mse(x) = sqrt.(sum(abs2, x)/length(x))
mxse(x) = sqrt.(maximum(x.^2))
mad(x) = maximum(abs.(x))
eopse(x,y) = sqrt.((prod(1 .+ x) .- prod(1 .+ y))^2)

eopse (generic function with 1 method)

In [10]:
function loss(m,x,y;func=mse)
    f = reshape(x[:,1:obs],N,obs,1,1)
    w = m(f) |> transpose
    r = x[:,obs+2:end]
    tmp = w*r
    rp = tmp .- tcost * w * (r .- tmp)
    if func != eopse
        return func(vec(rp) .- y)
    end
    return func(vec(rp), y)
end

loss (generic function with 1 method)

In [11]:
avgloss(model,ds;func=mse) = mean([loss(model,x,y,func=func) 
    for (x,y) in ds])

avgloss (generic function with 1 method)

## Logging

In [64]:
function report(tblogger, epoch)
    train = avgloss(model,zip(xtrain,ytrain),func=lossfn)
    test = avgloss(model,zip(xtest,ytest),func=lossfn)     
    #println("Epoch: $epoch   Train: $(train)   Test: $(test)")
    set_step!(tblogger, epoch)
    with_logger(tblogger) do
        @info "train" loss=train
        @info "test"  loss=test
    end
end

report (generic function with 1 method)

## Model

In [36]:
function convnet(h, w, ch1, ch2, k)
    out_conv_size = floor(Int, (h/k/k)) * floor(Int,(w/k/k)) * ch2
    model = Chain(Conv((k,k),1=>ch1,pad=SamePad()), 
            Dropout(0.5), 
            MaxPool((k,k)), 
            Conv((k,k),ch1=>ch2,pad=SamePad()),
            Dropout(0.5),
            MaxPool((k,k)), 
            Flux.flatten, 
            Dense(out_conv_size, 64, relu), 
            Dense(64,h, relu), 
            softmax)
    return model
end

convnet (generic function with 1 method)

In [21]:
function conv_block(cin,cout,k; p=0.5, act=relu)
    l = []
    kernel = (k,k)
    push!(l, Conv(kernel, cin => cout, act, pad=SamePad()))
    push!(l, BatchNorm(cout))
    push!(l, Dropout(p))
    push!(l, MaxPool(kernel))
    return l
end

conv_block (generic function with 1 method)

In [22]:
function make_model(w, h, c; k = 5, channels = [4,8], p = 0.5, act=relu)
    layers = []
    kernel = (k,k)
    conv = conv_block(c,channels[1],k,p=p, act=act)
    push!(layers, conv...)
    for (i, o) in zip(channels, channels[2:end])
        conv = conv_block(i,o,k,p=p, act=act)
        push!(layers, conv...)
    end
    conv_size = Flux.outputsize(layers, (w, h, c); padbatch=true)
    linear = Dense(prod(conv_size), h, relu)
    return Chain(layers..., Flux.flatten, linear, softmax)
end

make_model (generic function with 1 method)

## Training

In [53]:
epochs = 1000
η = 1e-3
λ = 1e-5
info_freq = 10
check_freq = 20

20

Choose loss function.

In [65]:
lossfn = mad

mad (generic function with 1 method)

In [71]:
savepath = joinpath("./log",String(Symbol(lossfn)))
tblogger = TBLogger(savepath, tb_overwrite)

TBLogger:
	- Log level     : Info
	- Current step  : 0
	- Output        : /home/balaji/projects/replicator/log/mad
	- open files    : 1


In [72]:
model = make_model(obs, N, 1, k=2, channels=[2,4])
opt = Flux.setup(OptimiserChain(WeightDecay(λ), ADAM(η)), model);

In [73]:
@showprogress for e in 1:epochs
    for (x,y) in zip(xtrain,ytrain)
        gs = gradient(m->loss(m,x,y,func=lossfn),model)
        Flux.update!(opt,model,gs[1])
    end
    (e % info_freq == 0) && report(tblogger, e)
    if e % check_freq == 0
        modelpath = joinpath(savepath,"model_epoch_$e.bson")
        BSON.@save modelpath model
    end
end

Progress: 100%|█████████████████████████████████████████| Time: 0:00:25
